In [1]:
#encoding=utf-8
import requests
import time
import json
from datetime import datetime, timedelta

def download(date):
    '''
    download data
    :param date:
    :return:
    '''
    url = "https://zgdypw.cn/pors/w/webStatisticsDatas/api/{}/searchDayBoxOffice".format(date)
    headers = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Host": "zgdypw.cn",
        "Pragma": "no-cache",
        "Referer": "https://zgdypw.cn/",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36"
    }
    proxies = {
        "http": "****",
        "https": "****",
    }

    response = requests.get(url, headers=headers, proxies=proxies)
    response.encoding = "utf-8"
    if response.status_code == 200:
        data = json.loads(response.test)["data"]
        dayBoxOffice = data["dayBoxOffice"]
        with open("dayBoxOffice.csv", "a+", encoding="utf-8") as file:
            file.write("{0}, {1}, {2}, {3}, {4}\n". format(
                dayBoxOffice["businessDay"],
                dayBoxOffice["cinemaCount"],
                dayBoxOffice["totalAudience"],
                dayBoxOffice["totalBoxoffices"],
                dayBoxOffice["totalSession"],
            ))
            
        top10CinemaChains = data["top10CinemaChains"]
        with open("top10CinemaChains.csv", "a+", encoding="utf-8") as file:
            for item in top10CinemaChains:
                file.write("{0}, {1}, {2}, {3}, {4}, {5}\n". format(
                    date,
                    item["cinemaChainName"],
                    item["dayAudience"],
                    item["daySession"],
                    item["rank"],
                    item["totalSales"],
                ))
                
        top10Cinemas = data["top10Cinemas"]
        with open("top10Cinemas.csv", "a+", encoding="utf-8") as file:
            for item in top10Cinemas:
                file.write("{0}, {1}, {2}, {3}, {4}, {5}\n". format(
                    date,
                    item["cinemaName"],
                    item["dayAudience"],
                    item["daySession"],
                    item["rank"],
                    item["totalSales"],
                ))
                
        top10Citys = data["top10Citys"]
        with open("top10Citys.csv", "a+", encoding="utf-8") as file:
            for item in top10Citys:
                file.write("{0}, {1}, {2}, {3}, {4}, {5}\n". format(
                    date,
                    item["cityName"],
                    item["dayAudience"],
                    item["daySession"],
                    item["rank"],
                    item["totalSales"],
                ))
                
        top10Films = data["top10Films"]
        with open("top10Films.csv", "a+", encoding="utf-8") as file:
            for item in top10Citys:
                file.write("{0}, {1}, {2}, {3}, {4}, {5}\n". format(
                    date,
                    item["filmName"],
                    item["dayAudience"],
                    item["daySession"],
                    item["rank"],
                    item["daySales"],
                    item["filmTotalSales"],
                ))
if __name__ == "__main__":
    start = datetime(year=2019, month=1, day=31)
    end = datetime.now()
    
    days = 0
    tem = start + timedelta(days=days)
    while tem <= end:
        print(tem.strftime("%Y-%m-%d"))
        download(tem.strftime("%Y-%m-%d"))
        days += 1
        tem = start + timedelta(days=days)
        

2019-01-31


ProxyError: HTTPSConnectionPool(host='zgdypw.cn', port=443): Max retries exceeded with url: /pors/w/webStatisticsDatas/api/2019-01-31/searchDayBoxOffice (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10f898ba8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known')))

In [ ]:
# 每日票房曲线
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import time

df = pd.read_csv("dayBoxOffice.csv")
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", inplace = True)

plt.figure(figsize = (20, 8), dpi = 80)
data = df["totalBoxoffice"]
_x = data.index
_x = [row.strftime("%Y-%m-%d") for row in _x]
_y = data.values

interval = 7
last = time.strftime("%Y-%m-%d", time.localtime(time.mktime(time.strptime(_x[-1], "%Y-%m-%d")) + interval * 24 * 60 *60))
plt.xticks(range(0, len(_x) + interval, interval), (_x + [last])[::interval], rotation=45)

plt.xlabel("日期")
plt.ylabel("票房（单位：万）")
plt.title("2019年电影每日票房")

plt.grid()
plt.show()

In [ ]:
# 每日的人次曲线
plt.figure(figsize=(20,8), dpi=80)

data = df["totalAudience"]
_x = data.index
_x = [row.strftime("%Y-%m-%d") for row in _x]
_y = data.values.astype(int)/10000
plt.plot(range(len(_x)), _y)

interval = 7
last = time.strftime("%Y-%m-%d", time.localtime(time.mktime(time.strptime(_x[-1], "%Y-%m-%d")) + interval * 24 * 60 *60))
plt.xticks(range(0, len(_x) + interval, interval), (_x + [last])[::interval], rotation=45)

plt.xlabel("日期")
plt.ylabel("人次（单位：万）")
plt.title("2019年电影每日观影人次图")

plt.grid()
plt.show()